In [1]:
import pandas as pd 
import plotly.graph_objects as go


# Load data assuming CSV files are in the same directory
distribution_centers = pd.read_csv("./data/distribution_centers.csv")
events = pd.read_csv("./data/events.csv")
inventory_items = pd.read_csv("./data/inventory_items.csv")
order_items = pd.read_csv("./data/order_items.csv")
orders = pd.read_csv("./data/orders.csv")
products = pd.read_csv("./data/products.csv")
users = pd.read_csv("./data/users.csv")

## EDA

In [2]:
inventory_items

,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,67971,13844,2022-07-02 07:09:20+00:00,2022-07-24 06:33:20+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
1,67972,13844,2023-12-20 03:28:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
2,67973,13844,2023-06-04 02:53:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
3,72863,13844,2021-10-16 22:58:52+00:00,2021-11-22 02:19:52+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
4,72864,13844,2021-08-07 16:33:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
...,...,...,...,...,...,...,...,...,...,...,...,...
490700,311018,25590,2021-12-07 12:47:00+00:00,NaN,10.39200,Underwear,ck one Men's Micro Slim Fit Boxer,Calvin Klein,24.00,Men,F220707C3AFE39D1779E0A34AF72BB7F,3
490701,451084,25590,2023-10-10 11:57:57+00:00,2023-11-27 11:28:57+00:00,10.39200,Underwear,ck one Men's Micro Slim Fit Boxer,Calvin Klein,24.00,Men,F220707C3AFE39D1779E0A34AF72BB7F,3
490702,451085,25590,2020-11-20 05:55:00+00:00,NaN,10.39200,Underwear,ck one Men's Micro Slim Fit Boxer,Calvin Klein,24.00,Men,F220707C3AFE39D1779E0A34AF72BB7F,3
490703,486057,25590,2023-02-26 22:35:12+00:00,2023-04-23 01:24:12+00:00,10.39200,Underwear,ck one Men's Micro Slim Fit Boxer,Calvin Klein,24.00,Men,F220707C3AFE39D1779E0A34AF72BB7F,3


### Merge Datasets - order_items and inventory_items

In [3]:
# Rename the column in inventory_items to match the column in order_items
inventory_items.rename(columns={'id': 'inventory_item_id'}, inplace=True)


In [4]:
# Merge DataFrames
merged_df = pd.merge(order_items, inventory_items, on='inventory_item_id', how='inner')

# Check for missing values after merge
print(merged_df.isnull().sum())

id                                     0
order_id                               0
user_id                                0
product_id_x                           0
inventory_item_id                      0
status                                 0
created_at_x                           0
shipped_at                         63478
delivered_at                      117918
returned_at                       163527
sale_price                             0
product_id_y                           0
created_at_y                           0
sold_at                           181759
cost                                   0
product_category                       0
product_name                          10
product_brand                        149
product_retail_price                   0
product_department                     0
product_sku                            0
product_distribution_center_id         0
dtype: int64


In [5]:
# Define a function to remove suffixes and add prefixes
def rename_columns(col):
    if col.endswith('_x'):
        return 'order_' + col[:-2]  # Add 'order_' prefix and remove the '_x' suffix
    elif col.endswith('_y'):
        return 'inventory_' + col[:-2]  # Add 'inventory_' prefix and remove the '_y' suffix
    return col

# Rename columns
merged_df.columns = [rename_columns(col) for col in merged_df.columns]


In [6]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181759 entries, 0 to 181758
Data columns (total 22 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              181759 non-null  int64  
 1   order_id                        181759 non-null  int64  
 2   user_id                         181759 non-null  int64  
 3   order_product_id                181759 non-null  int64  
 4   inventory_item_id               181759 non-null  int64  
 5   status                          181759 non-null  object 
 6   order_created_at                181759 non-null  object 
 7   shipped_at                      118281 non-null  object 
 8   delivered_at                    63841 non-null   object 
 9   returned_at                     18232 non-null   object 
 10  sale_price                      181759 non-null  float64
 11  inventory_product_id            181759 non-null  int64  
 12  inventory_create

In [7]:
date_columns = ['order_created_at', 'shipped_at', 'delivered_at', 'returned_at', 'inventory_created_at']

# Convert date columns to datetime format with ISO8601
for column in date_columns:
    merged_df[column] = pd.to_datetime(merged_df[column], format='ISO8601')

# Print sample data
merged_df

,id,order_id,user_id,order_product_id,inventory_item_id,status,order_created_at,shipped_at,delivered_at,returned_at,...,inventory_created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaT,NaT,NaT,...,2021-09-09 06:17:00+00:00,NaN,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaT,...,2023-06-05 15:24:00+00:00,NaN,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaT,...,2020-04-12 14:36:00+00:00,NaN,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaT,NaT,...,2020-01-24 05:47:00+00:00,NaN,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaT,NaT,...,2022-09-13 16:58:00+00:00,NaN,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181754,9674,6679,5325,24447,26095,Returned,2020-12-23 00:05:01+00:00,2020-12-25 22:54:00+00:00,2020-12-28 07:08:00+00:00,2020-12-30 22:43:00+00:00,...,2021-10-07 09:37:00+00:00,NaN,404.595001,Outerwear & Coats,Darla,Alpha Industries,999.00,Men,1CE5E897CDA6AEB211DFFE8D514F4365,5
181755,7801,5416,4283,24447,21078,Shipped,2022-01-12 23:51:07+00:00,2022-01-10 03:42:00+00:00,NaT,NaT,...,2023-09-16 00:38:00+00:00,NaN,404.595001,Outerwear & Coats,Darla,Alpha Industries,999.00,Men,1CE5E897CDA6AEB211DFFE8D514F4365,5
181756,62986,43364,34691,23546,169937,Shipped,2023-02-26 00:38:43+00:00,2023-02-25 06:53:00+00:00,NaT,NaT,...,2021-11-03 11:45:00+00:00,NaN,482.516999,Shorts,Alpha Industries Rip Stop Short,Alpha Industries,999.00,Men,C1AA157F9E8516370E3345F226EF62FC,5
181757,106577,73418,58623,24447,287560,Shipped,2023-11-12 02:41:02+00:00,2023-11-14 20:44:00+00:00,NaT,NaT,...,2022-11-04 09:27:00+00:00,NaN,404.595001,Outerwear & Coats,Darla,Alpha Industries,999.00,Men,1CE5E897CDA6AEB211DFFE8D514F4365,5


In [8]:
# Calculate order duration
merged_df['order_duration'] = merged_df['shipped_at'] - merged_df['order_created_at']

# Calculate shipping duration
merged_df['shipping_duration'] = merged_df['delivered_at'] - merged_df['shipped_at']

# Calculate return duration
merged_df['return_duration'] = merged_df['returned_at'] - merged_df['delivered_at']

# Calculate total revenue
merged_df['total_revenue'] = merged_df['sale_price'] * merged_df['order_product_id']

# Calculate total cost
merged_df['total_cost'] = merged_df['cost'] * merged_df['order_product_id']

# Calculate total profit
merged_df['total_profit'] = merged_df['total_revenue'] - merged_df['total_cost']

In [9]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181759 entries, 0 to 181758
Data columns (total 28 columns):
 #   Column                          Non-Null Count   Dtype              
---  ------                          --------------   -----              
 0   id                              181759 non-null  int64              
 1   order_id                        181759 non-null  int64              
 2   user_id                         181759 non-null  int64              
 3   order_product_id                181759 non-null  int64              
 4   inventory_item_id               181759 non-null  int64              
 5   status                          181759 non-null  object             
 6   order_created_at                181759 non-null  datetime64[ns, UTC]
 7   shipped_at                      118281 non-null  datetime64[ns, UTC]
 8   delivered_at                    63841 non-null   datetime64[ns, UTC]
 9   returned_at                     18232 non-null   datetime64[ns, UTC]
 

In [10]:
# Calculate the time it took for each order to be returned after it was created
merged_df['return_time'] = merged_df['returned_at'] - merged_df['order_created_at']

# Calculate the return rate
total_orders = len(merged_df)
returned_orders = merged_df['returned_at'].notna().sum()
return_rate = returned_orders / total_orders

# Print the return rate
print("Return Rate:", return_rate)

Return Rate: 0.1003086504657266


## Sales overview

In [11]:
#Total number of Brands and Products
number_of_brands = products['brand'].nunique()
number_of_products = products['id'].nunique()

print(f'The Number of Brands: {number_of_brands},\nThe Number of Products: {number_of_products}')

The Number of Brands: 2756,
The Number of Products: 29120


In [12]:
#How many brands and items currently on sale(instock)
instock_brand = inventory_items['product_brand'].nunique()
instock_products = inventory_items['product_id'].nunique()

print(f'The Number of Brands In Stock: {instock_brand},\nThe Number of Products In Stock: {instock_products}')

The Number of Brands In Stock: 2752,
The Number of Products In Stock: 29046


## Merge complete orders with products 

In [13]:
import plotly.graph_objects as go

# Merge complete orders with products 
orders_merged = pd.merge(order_items, products, left_on='product_id', right_on='id', how='left')
orders_merged['profit'] = orders_merged['sale_price'] - orders_merged['cost']

In [14]:
orders

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaN,NaN,NaN,3
1,60,44,Cancelled,F,2023-01-20 02:12:00+00:00,NaN,NaN,NaN,1
2,64,46,Cancelled,F,2021-12-06 09:11:00+00:00,NaN,NaN,NaN,1
3,89,65,Cancelled,F,2020-08-13 09:58:00+00:00,NaN,NaN,NaN,1
4,102,76,Cancelled,F,2023-01-17 08:17:00+00:00,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...
125221,125196,99972,Shipped,M,2022-06-25 11:56:00+00:00,NaN,2022-06-28 08:24:00+00:00,NaN,1
125222,125199,99975,Shipped,M,2022-07-20 02:19:00+00:00,NaN,2022-07-22 11:08:00+00:00,NaN,4
125223,125208,99983,Shipped,M,2022-03-14 15:06:00+00:00,NaN,2022-03-17 05:15:00+00:00,NaN,1
125224,125210,99983,Shipped,M,2023-03-01 15:06:00+00:00,NaN,2023-03-02 10:50:00+00:00,NaN,2


In [15]:
order_items

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaN,NaN,NaN,0.02
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaN,0.02
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,0.02
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaN,NaN,0.02
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaN,NaN,0.02
...,...,...,...,...,...,...,...,...,...,...,...
181754,9674,6679,5325,24447,26095,Returned,2020-12-23 00:05:01+00:00,2020-12-25 22:54:00+00:00,2020-12-28 07:08:00+00:00,2020-12-30 22:43:00+00:00,999.00
181755,7801,5416,4283,24447,21078,Shipped,2022-01-12 23:51:07+00:00,2022-01-10 03:42:00+00:00,NaN,NaN,999.00
181756,62986,43364,34691,23546,169937,Shipped,2023-02-26 00:38:43+00:00,2023-02-25 06:53:00+00:00,NaN,NaN,999.00
181757,106577,73418,58623,24447,287560,Shipped,2023-11-12 02:41:02+00:00,2023-11-14 20:44:00+00:00,NaN,NaN,999.00


In [16]:
products.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1


In [17]:
orders_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181759 entries, 0 to 181758
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id_x                    181759 non-null  int64  
 1   order_id                181759 non-null  int64  
 2   user_id                 181759 non-null  int64  
 3   product_id              181759 non-null  int64  
 4   inventory_item_id       181759 non-null  int64  
 5   status                  181759 non-null  object 
 6   created_at              181759 non-null  object 
 7   shipped_at              118281 non-null  object 
 8   delivered_at            63841 non-null   object 
 9   returned_at             18232 non-null   object 
 10  sale_price              181759 non-null  float64
 11  id_y                    181759 non-null  int64  
 12  cost                    181759 non-null  float64
 13  category                181759 non-null  object 
 14  name                

In [18]:
orders_merged

,id_x,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,...,id_y,cost,category,name,brand,retail_price,department,sku,distribution_center_id,profit
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaN,NaN,NaN,...,14235,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,0.011700
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaN,...,14235,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,0.011700
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,...,14235,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,0.011700
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaN,NaN,...,14235,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,0.011700
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaN,NaN,...,14235,0.008300,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1,0.011700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181754,9674,6679,5325,24447,26095,Returned,2020-12-23 00:05:01+00:00,2020-12-25 22:54:00+00:00,2020-12-28 07:08:00+00:00,2020-12-30 22:43:00+00:00,...,24447,404.595001,Outerwear & Coats,Darla,Alpha Industries,999.00,Men,1CE5E897CDA6AEB211DFFE8D514F4365,5,594.404999
181755,7801,5416,4283,24447,21078,Shipped,2022-01-12 23:51:07+00:00,2022-01-10 03:42:00+00:00,NaN,NaN,...,24447,404.595001,Outerwear & Coats,Darla,Alpha Industries,999.00,Men,1CE5E897CDA6AEB211DFFE8D514F4365,5,594.404999
181756,62986,43364,34691,23546,169937,Shipped,2023-02-26 00:38:43+00:00,2023-02-25 06:53:00+00:00,NaN,NaN,...,23546,482.516999,Shorts,Alpha Industries Rip Stop Short,Alpha Industries,999.00,Men,C1AA157F9E8516370E3345F226EF62FC,5,516.483001
181757,106577,73418,58623,24447,287560,Shipped,2023-11-12 02:41:02+00:00,2023-11-14 20:44:00+00:00,NaN,NaN,...,24447,404.595001,Outerwear & Coats,Darla,Alpha Industries,999.00,Men,1CE5E897CDA6AEB211DFFE8D514F4365,5,594.404999


In [19]:
# Format the date columns
orders_merged[['created_at', 'shipped_at', 'delivered_at']] = \
    orders_merged[['created_at', 'shipped_at', 'delivered_at']].apply(lambda col: pd.to_datetime(col, errors='coerce', format='mixed'))

# Include only completed orders
orders_complete_merged = orders_merged.loc[orders_merged['status'] == 'Complete'].copy()

# Include orders after refund period
fourteen_period = orders_complete_merged['delivered_at'].max() - pd.DateOffset(days=14)
orders_complete_merged = orders_complete_merged[orders_complete_merged['delivered_at'] <= fourteen_period]

# Calculate total revenue, cost of goods, and profits
total_revenue = round(orders_complete_merged['sale_price'].sum(), 2)
total_cost = round(orders_complete_merged['cost'].sum(), 2)
total_profit = round(orders_complete_merged['profit'].sum(), 2)



In [20]:
# Print results
print("Total Revenue: $", total_revenue)
print("Total Cost of Goods: $", total_cost)
print("Gross Profit: $", total_profit)

Total Revenue: $ 2565780.67
Total Cost of Goods: $ 1234037.65
Gross Profit: $ 1331743.03


In [21]:
# Plot the table
fig = go.Figure(data=[go.Table(
    header=dict(values=['Total Sales Revenue', 'Total Cost of Goods', 'Total Gross Profit'],
                fill_color='darkgray',
                align='left'),  
    cells=dict(values=[total_revenue, total_cost, total_profit],
               fill_color='lightgrey',
               align='left'))])

fig.update_layout(font=dict(color='black'),
                  title_text='Total Revenue, Cost of Goods, and Gross Profit')

fig.show()

## Sales Performance by Product Category and Brand:


In [22]:
# Calculate total revenue by product category
revenue_by_category = orders_merged.groupby('category')['profit'].sum().reset_index()

# Sort by total revenue in descending order
revenue_by_category = revenue_by_category.sort_values(by='profit', ascending=False)

# Print sales performance by product category
print("Sales Performance by Product Category:")
print(revenue_by_category)


Sales Performance by Product Category:
                         category         profit
11              Outerwear & Coats  721875.075552
7                           Jeans  582912.002207
22                       Sweaters  437260.450999
21            Suits & Sport Coats  398930.048650
23                           Swim  316596.397142
5   Fashion Hoodies & Sweatshirts  312268.251792
17                 Sleep & Lounge  277012.735385
1                          Active  270137.246519
15                         Shorts  255751.936677
4                         Dresses  254917.227952
0                     Accessories  250189.657921
12                          Pants  236164.650834
24                    Tops & Tees  216371.354568
6                       Intimates  215653.440392
2               Blazers & Jackets  181379.038838
10                      Maternity  148312.282145
25                      Underwear  107579.699544
13                 Pants & Capris   88753.926371
14                           P

In [23]:
import plotly.express as px

# Plot sales performance by product category
fig = px.bar(revenue_by_category, x='category', y='profit', title='Sales Performance by Product Category')
fig.update_xaxes(title='Product Category')
fig.update_yaxes(title='Total Revenue')
fig.show()


In [32]:
# Calculate total revenue by product brand
revenue_by_brand = orders_merged.groupby('brand')['profit'].sum().reset_index()

# Sort by total revenue in descending order
revenue_by_brand = revenue_by_brand.sort_values(by='profit', ascending=False)

# Print sales performance by product brand
print("Sales Performance by Product brand:")
print(revenue_by_brand)


Sales Performance by Product brand:
                  brand         profit
447        Calvin Klein  110823.315214
701              Diesel   99194.947899
465            Carhartt   97820.019780
29    7 For All Mankind   90264.142211
2486      True Religion   85944.728796
...                 ...            ...
805                Eros       4.690500
833           Extenders       3.683750
2200       Sock Company       2.649600
1544        Made in USA       1.250480
2736            marshal       0.070200

[2752 rows x 2 columns]


In [35]:
# Create a table figure
fig = go.Figure(data=[go.Table(
    header=dict(values=['Product Brand', 'Total Revenue'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[revenue_by_brand['brand'], revenue_by_brand['profit']],
               fill_color='lavender',
               align='left'))
])

# Update layout
fig.update_layout(title='Sales Performance by Product Brand')
fig.update_xaxes(title='Product Brand')
fig.update_yaxes(title='Total Revenue')

# Show the table
fig.show()